In [109]:
import csv
import math
from random import random

In [110]:
train_data = csv.reader(open("mnist_test.csv", mode="r"))
def GetSample(data):
    length = len(data)
    for i in range(length):
        data[i] = float(data[i]) / 255.0
    return data

In [111]:
class Network:
    def __init__(self, sizes) -> None:
        self.cost_array = []
        self.output_layer = Network_Layer(sizes[len(sizes) - 1], sizes[len(sizes) - 2])
        self.hidden_layers = []
        for i in range(1, len(sizes) - 1):
            self.hidden_layers.append(Network_Layer(sizes[i], sizes[i - 1]))
        self.layers = [ *self.hidden_layers, self.output_layer ]
    
    def forward_pass(self, input_data):
        print("Beginning forward pass...")
        layer_output = input_data
        for layer in self.layers:
            layer_output = layer.processData(layer_output)
            print(layer_output)
        return self.SoftMax(layer_output)
    
    def backward_pass(self):
        pass

    def SoftMax(self, input_data):
        max = input_data[0]
        for neuron in input_data:
            max = neuron if neuron > max else max
        
        exponented_layer = []
        for neuron in input_data:
            exponented_layer.append(math.exp(neuron - max))

        Layer_Sum = sum(exponented_layer)

        normalized_layer = []
        for neuron in exponented_layer:
            normalized_layer.append(neuron/Layer_Sum)
        return normalized_layer

    def UpdateNetworkCost(self, answer):
        neurons = self.output_layer.neurons
        for i in range(len(neurons)):
            value = 1.00 if answer == i else 0.00
            self.cost_array.append((neurons[i].last_output - value) ** 2)
        return sum(self.cost_array)

            


In [112]:
class Network_Layer:
    def __init__(self, num_neurons, input_size) -> None:
        self.weights = []
        self.neurons = []
        self.connections = input_size
        for i in range(num_neurons):
            self.weights.append(self.GenerateWeights())
            self.neurons.append(Neuron())
            
    def GenerateWeights(self):
        weights = []
        for i in range(self.connections):
            weights.append(10.0*random()-5.0)
        return weights

    def processData(self, input_data):
        outputs = []
        for i in range(len(self.neurons)):
            self.neurons[i].ProcessInputs(input_data, self.weights[i])
            outputs.append(self.neurons[i].last_output)
        self.last_output = outputs
        return self.last_output


In [113]:
class Neuron:
    def __init__(self) -> None:
        self.bias = 10.0*random()-5.0
    def ProcessInputs(self, input_layer, network_weights):
        total = 0
        for i in range(len(network_weights)):
            total += input_layer[i] * network_weights[i]
        total = self.Activation_Sigmoid(total + self.bias)
        self.last_output = total
        return self.last_output
    
    def Activation_ReLU(self, input):
        return input if input > 0 else 0

    def Activation_Sigmoid(self, input):
        return 1.00/(1.00+math.exp(-input))


In [114]:
network = Network(sizes=[784, 16, 16, 10])
for layer in network.layers:
    print("Neurons: ", len(layer.neurons), "Inputs: ", len(layer.weights[0]))

data = GetSample(train_data.__next__())
processed = network.forward_pass(data[1:])
print(processed)
print(sum(processed))
print(network.UpdateNetworkCost(data[0]))

Neurons:  16 Inputs:  784
Neurons:  16 Inputs:  16
Neurons:  10 Inputs:  16
Beginning forward pass...
[0.9999999999999827, 1.2864738853460821e-06, 7.915257636154296e-06, 0.9999993465832671, 0.00012493581440316284, 5.410132823277039e-10, 0.9999992762075344, 0.9999993145946385, 2.6726585015399177e-07, 0.13224431344387508, 7.42458247654294e-07, 0.00012871223194438965, 0.9975582432053313, 1.1668070104973565e-11, 0.002925247643187989, 0.27139851270351467]
[0.0012731969743574284, 0.9992442516143283, 0.887120853715098, 0.9774273568940416, 0.00016718776504178303, 0.9672786589228813, 0.22152530821726255, 0.9992511726264997, 0.9989211145066419, 0.999946658504278, 1.6765710445721773e-05, 0.9893513185037487, 0.18479868311370168, 7.14987912373912e-06, 0.7792167947118078, 0.5697610112454004]
[0.919109686793442, 0.04045929332316926, 0.44319275187127904, 5.456373929971393e-05, 0.9999996604051428, 0.9868115932105235, 0.0006937177176154371, 0.9995969126037769, 1.1298658461367486e-06, 0.00193074124595247